# COVID-19 Trend Tracker

This dashboard displays COVID-19 data in an interactive manner. The graph displays the number of new cases and tests over time. You can choose to view this data on either a linear or logarithmic by selecting the desired option under "Scale". This allows for a more nuanced exploration of the data trends, The "Data Column" dropdown menu allows you to switch between viewing new cases or test data.

Click the "Refresh Data" button to make sure you're looking at the most recent data.This button fetches the latest data from the API and updates the graph accordingly. If the API connection fails, the graph will shift to using the most current available local data.

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [ ]:
# open the 'data.json' file and load its contents into 'jsondata'.
with open("data.json", "rt") as INFILE:
    jsondata = json.load(INFILE)


In [ ]:
def parse_date(datestring):#parse_date function
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):#create a DataFrame with new COVID cases and tests by date.
    datalist = rawdata['data']
    dates = [dictionary['date'] for dictionary in datalist]
    dates.sort()
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])
    # create a DataFrame with dates
    index = pd.date_range(startdate, enddate, freq='D')
    casetestdf = pd.DataFrame(index=index, columns=['newCases', 'tests'])
    for entry in datalist:# fill the DataFrame with case and test data
        date = parse_date(entry['date'])
        for column in ['newCases', 'tests']:
            if pd.isna(casetestdf.loc[date, column]):
                value = (float(entry[column]) if entry[column] != None else 0.0)
                casetestdf.loc[date, column] = value
    return casetestdf

df = wrangle_data(jsondata) #initialize df

In [ ]:
def access_api():#access the API
    filters = [
        'areaType=nation',
        'areaName=England'
    ]
    structure = {
        "date": "date",
        "newCases": "newCasesBySpecimenDateRollingSum",
        "tests": "uniquePeopleTestedBySpecimenDateRollingSum"
    }
    
    try:
        api = Cov19API(filters=filters, structure=structure)
        data = api.get_json()
        return data  # If successful, return the data
    except Exception as e:#if API access fails and return None
        print(f"An error occurred while fetching data from the API: {e}")
        return None  # Return None to indicate that the fetch failed

In [ ]:
def api_button_callback(button):
    global df#global df variable
    try:# try to get data from the API
        apidata = access_api()
        if apidata is not None:  # check if API call was successful
            df = wrangle_data(apidata)
            refresh_graph()# update the graph
            button.icon = "check"# show success icon on button
        else:   # if API call failed, use the local json data
            df = wrangle_data(jsondata)
            refresh_graph()            
            button.icon = "times"# show failure icon on button
            print("Failed to fetch data from the API. Using canned data.\nThe 'canned' data are not overwritten and nothing crashes")
    except Exception as e:
        button.icon = "times"
        print(f"Using canned data.\nThe 'canned' data are not overwritten and nothing crashes: {e}")
    finally:
        pass

# Create a button for updating data
apibutton=wdg.Button(
    description='Refresh Data', # text on the button
    disabled=False,
    button_style='info', #color
    tooltip="Click to refresh data",
    icon='sync-alt'
)

# Link the button to the api_button_callback
apibutton.on_click(api_button_callback)
display(apibutton)


In [ ]:

def df_graph(gcol, gscale):#  Function to plot the graph
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    if len(gcol)>0:
        df[[gcol]].plot(kind='line', logy=logscale)  
        plt.show() 
    else:
        print("Selected data column is not in the DataFrame or no column is selected.")


#select data column for plotting
series = wdg.Dropdown(
    options=['newCases', 'tests'],
    value='newCases',
    description='Data Column:',
    disabled=False
)
#buttons to select scale type
scale = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls = wdg.HBox([series, scale])
graph = wdg.interactive_output(df_graph, {'gcol': series, 'gscale': scale})# Create an interactive graph output
display(controls,graph)

def refresh_graph():# Function to force the graph to update
    current_scale = scale.value
    other_scale = 'linear' if current_scale == 'log' else 'log'
    scale.value = other_scale  
    scale.value = current_scale  